In [1]:
import json
import pandas as pd
import numpy as np

In [5]:
from yfinance3 import YFinance3
import json

SYMBOLS = ['INTU','CDNS']

for symbol in SYMBOLS:
    data = YFinance3(symbol)
    file_name = f'{symbol}.json'
    with open(file_name, 'w') as file:
        json.dump(data.info, file)
    print('saved to {}'.format(file_name))

saved to INTU.json
saved to CDNS.json


In [6]:
SYMBOLS = ['INTU','CDNS']

data = {
    'Symbol': [],
    'Name': [],
    'Industry': [],
    'EPS (fwd)': [],
    'P/E (fwd)': [],
    'PEG': [],
    'FCFY' : [],
    'PB': [],
    'ROE' : [],
    'P/S (trail)': [],
    'DPR' : [],
    'DY' : [],
    'CR' : [],
    'Beta': [],
    'Price': [],
    '52w Low': [],
    '52w High': []
    }

In [7]:
def load_data(json_data):
    data['Symbol'].append(json_data['symbol'])
    data['Name'].append(json_data['longName'])
    data['Industry'].append(json_data['industry'])
    data['Price'].append(json_data['currentPrice'])

    # Could be that some indicators are not available; use NaN if this is the case
    
    if 'forwardEps' in json_data:
        data['EPS (fwd)'].append(json_data['forwardEps'])
    else:
        data['EPS (fwd)'].append(np.nan)
        
    if 'forwardPE' in json_data:
        data['P/E (fwd)'].append(json_data['forwardPE'])
    else:
        data['P/E (fwd)'].append(np.nan)
        
    if 'pegRatio' in json_data:
        data['PEG'].append(json_data['pegRatio'])
    else:
        data['PEG'].append(np.nan)

    if ('freeCashflow' in json_data) and ('marketCap' in json_data):
        fcfy = (json_data['freeCashflow']/json_data['marketCap']) * 100
        data['FCFY'].append(round(fcfy, 2))
    else:
        data['FCFY'].append(np.nan)

    if 'priceToBook' in json_data:
        data['PB'].append(json_data['priceToBook'])
    else:
        data['PB'].append(np.nan)

    if 'returnOnEquity' in json_data:
        data['ROE'].append(json_data['returnOnEquity'])
    else:
        data['ROE'].append(np.nan)
        
    if 'priceToSalesTrailing12Months' in json_data:
        data['P/S (trail)'].append(json_data['priceToSalesTrailing12Months'])
    else:
        data['P/S (trail)'].append(np.nan)

    data['DPR'].append(json_data['payoutRatio'] * 100)

    if 'dividendYield' in json_data:
        data['DY'].append(json_data['dividendYield'])
    else:
        data['DY'].append(0.0)

    if 'beta' in json_data:
        data['Beta'].append(json_data['beta'])
    else:
        data['Beta'].append(np.nan)

    if 'currentRatio' in json_data:
        data['CR'].append(json_data['currentRatio'])
    else:
        data['CR'].append(np.nan)

    if 'fiftyTwoWeekLow' in json_data:
        data['52w Low'].append(json_data['fiftyTwoWeekLow'])
    else:
        data['52w Low'].append(np.nan)
        
    if 'fiftyTwoWeekHigh' in json_data:    
        data['52w High'].append(json_data['fiftyTwoWeekHigh'])
    else:
        data['52w High'].append(np.nan)

In [8]:
for symbol in SYMBOLS:
    file_name = f'{symbol}.json'    
    try:
        with open(file_name, 'r') as file:
            load_data(json.load(file))
    except FileNotFoundError:
        print(f"File '{file_name}' not found.")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON data: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [9]:
df = pd.DataFrame(data)

df_exceptions = df[df.isna().any(axis=1)]

df=df.dropna()

df.reset_index(drop=True, inplace=True)

df['52w Range'] = ((df['Price'] - df['52w Low'])/(df['52w High'] - df['52w Low']))*100

df_exceptions

,Symbol,Name,Industry,EPS (fwd),P/E (fwd),PEG,FCFY,PB,ROE,P/S (trail),DPR,DY,CR,Beta,Price,52w Low,52w High


In [10]:
def make_pretty(styler):
    styler.format({'EPS (fwd)': '${:.2f}', 'P/E (fwd)': '{:.2f}', 'PEG': '{:.2f}',
                   'FCFY': '{:.2f}%', 'PB' : '{:.2f}', 'ROE' : '{:.2f}', 'P/S (trail)': '{:.2f}',
                   'DPR': '{:.2f}%', 'DY': '{:.2f}%', 'CR' : '{:.2f}', 'Beta': '{:.2f}', '52w Low': '${:.2f}',
                   'Price': '${:.2f}', '52w High': '${:.2f}', '52w Range': '{:.2f}%'
                  })
    styler.bar(subset = ['52w Range'], align = "mid", color = ["salmon", "cornflowerblue"])

    styler.set_properties(**{'border': '0.1px solid black'})

    styler.background_gradient(subset=['EPS (fwd)'], cmap='Greens')
    styler.background_gradient(subset=['P/E (fwd)'], cmap='Greens')
    styler.background_gradient(subset=['PEG'], cmap='Greens')
    styler.background_gradient(subset=['FCFY'], cmap='Greens')
    styler.background_gradient(subset=['PB'], cmap='Greens')
    styler.background_gradient(subset=['ROE'], cmap='Greens')
    styler.background_gradient(subset=['P/S (trail)'], cmap='Greens')
    styler.background_gradient(subset=['DPR'], cmap='Greens')
    styler.background_gradient(subset=['DY'], cmap='Greens')
    styler.background_gradient(subset=['CR'], cmap='Greens')

    styler.hide(axis='index')

    styler.set_tooltips(
        ttips, css_class='tt-add',
        props=[
            ('visibility', 'hidden'),
            ('position', 'absolute'),
            ('background-color', 'salmon'),
            ('color', 'black'),
            ('z-index', 1),
            ('padding', '3px 3px'),
            ('margin', '2px')
        ]
    )
    styler.set_properties(subset=['Symbol', 'Name', 'Industry'], **{'text-align': 'left'})
    return styler

In [11]:
def populate_tt(df, tt_data, col_name):
    stats = df[col_name].describe()
    
    per25 = round(stats.loc['25%'], 2)
    per50 = round(stats.loc['50%'], 2)
    per75 = round(stats.loc['75%'], 2)

    pos = df.columns.to_list().index(col_name)
    
    for index, row in df.iterrows():
        pe = row[col_name]
        if pe == stats.loc['min']:
            tt_data[index][pos] = 'Lowest'
        elif pe == stats.loc['max']:
            tt_data[index][pos] = 'Hightest'
        elif pe <= per25:
            tt_data[index][pos] = '25% of companies under {}'.format(per25)
        elif pe <= per50:
            tt_data[index][pos] = '50% of companies under {}'.format(per50)
        elif pe <= per75:
            tt_data[index][pos] = '75% of companies under {}'.format(per75)
        else:
            tt_data[index][pos] = '25% of companies over {}'.format(per75)   

In [12]:
tt_data = [['' for x in range(len(df.columns))] for y in range(len(df))]

populate_tt(df, tt_data, 'EPS (fwd)')
populate_tt(df, tt_data, 'P/E (fwd)')
populate_tt(df, tt_data, 'PEG')
populate_tt(df, tt_data, 'FCFY')
populate_tt(df, tt_data, 'PB')
populate_tt(df, tt_data, 'ROE')
populate_tt(df, tt_data, 'P/S (trail)')
populate_tt(df, tt_data, 'DPR')
populate_tt(df, tt_data, 'DY')
populate_tt(df, tt_data, 'CR')

ttips = pd.DataFrame(data=tt_data, columns=df.columns, index=df.index)

df.style.pipe(make_pretty).set_caption('Fundamental Indicators').set_table_styles(
    [{'selector': 'th.col_heading', 'props': 'text-align: center'},
     {'selector': 'caption', 'props': [('text-align', 'center'),
                                       ('font-size', '11pt'), ('font-weight', 'bold')]}])

Symbol,Name,Industry,EPS (fwd),P/E (fwd),PEG,FCFY,PB,ROE,P/S (trail),DPR,DY,CR,Beta,Price,52w Low,52w High,52w Range
INTU,Intuit Inc.,Software - Application,$17.06,36.73,3.14,2.45%,10.33,0.16,11.89,35.49%,0.01%,1.25,1.21,$626.57,$370.62,$628.33,99.32%
CDNS,"Cadence Design Systems, Inc.",Software - Application,$5.89,46.96,3.01,1.31%,24.16,0.33,19.19,0.00%,0.00%,1.44,1.05,$276.57,$154.88,$279.33,97.78%
